In [131]:
import cv2
import pandas as pd
import numpy as np

In [107]:
img = cv2.imread("./data/masks/0363100012175978_4.380233.png")
cv2.imshow("Display window", img)
k = cv2.waitKey(0) # Wait for a keystroke in the window

In [132]:
image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Convert the image to HSV for better color segmentation
hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)

# save CSV to read the color values of the image 
hsv_2d = hsv.reshape((-1,3))
df = pd.DataFrame(hsv_2d)
tiles_csv_path = './data/junk/hsv.csv'
df.to_csv(tiles_csv_path, index=False, header=False)

'not integrated in the code, just read from the file'

# retrieved color values
sky_val = np.array([0,0,255])
cl_fac_val = np.array([60,185,160])
window_fac_val = np.array([102,211,180])
door_fac_val = np.array([14,241,255])

    # # add margin
    # lower_val = window_fac_val -1
    # upper_val = window_fac_val +1

mask = cv2.inRange(hsv, window_fac_val, window_fac_val)

#save image
cv2.imwrite("data/cv2_window_mask.png", mask)

# show image 
    # cv2.imshow("img", mask)
    # cv2.waitKey(0)
    # cv.destroyAllWindows()

True

In [127]:
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # contour = contours[0]
    # print(type(contours))
    # print(cv2.boundingRect(contour))

    # Draw the contours on the original image for visualization
    # image_contours = image.copy()
    # cv2.drawContours(image_contours, contours, -1, (0, 255, 0), 3)

# Calculate the height of each square's border
borders = []

for contour in contours:
    borders.append(cv2.boundingRect(contour))

print(borders)

[(106, 738, 32, 73), (58, 735, 33, 77), (17, 733, 29, 28), (58, 661, 35, 58), (15, 660, 32, 58), (105, 658, 34, 62), (106, 588, 34, 48), (13, 586, 33, 48), (59, 584, 34, 52), (60, 526, 31, 42)]


In [ ]:
# Approximate contours to polygons + get bounding rects and circles
contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
# centers = [None]*len(contours)
# radius = [None]*len(contours)
for i, c in enumerate(contours):
    contours_poly[i] = cv2.approxPolyDP(c, 3, True)
    boundRect[i] = cv2.boundingRect(contours_poly[i])
    # centers[i], radius[i] = cv.minEnclosingCircle(contours_poly[i])

drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)

# Draw polygonal contour + bonding rects + circles
for i in range(len(contours)):
    color = (0,0,255)
    # cv2.drawContours(drawing, contours_poly, i, color)
    cv2.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)


In [166]:
x,y,w,h = cv2.boundingRect(contour)
# cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

rect = cv2.minAreaRect(contour)
box = cv2.boxPoints(rect)
box = np.int0(box)
show = cv2.drawContours(img,[box],0,(0,0,255),2)
cv2.imshow("show", show)
k = cv2.waitKey(0) # Wait for a keystroke in the window

<ipython-input-166-6d46c01a66fd>:6: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


In [128]:
#list heights of the upper and lower corner of each window
y_bounds = []
for border in borders:
    y_bound = []
    y_bound.append(border[1])
    lower_corner = border[1] - border[3]
    y_bound.append(lower_corner)
    y_bounds.append(y_bound)
print(y_bounds)

[[738, 665], [735, 658], [733, 705], [661, 603], [660, 602], [658, 596], [588, 540], [586, 538], [584, 532], [526, 484]]


In [129]:
# first cluster-tests     
print(y_bounds)
if y_bounds[1][1] <= y_bounds[0][0] and y_bounds[0][1] <= y_bounds[1][0]: 
    print("yay!")
else: 
    print("nay..")    

[[738, 665], [735, 658], [733, 705], [661, 603], [660, 602], [658, 596], [588, 540], [586, 538], [584, 532], [526, 484]]
yay!


In [133]:
window_clusters = []
for index, window_1 in enumerate(y_bounds): 
    window_clusters.append(index)
    cluster = []
    for window_2 in y_bounds: 
        if window_2[1] <= window_1[0] and window_1[1] <= window_2[0]: 
            # print("yay!")
            cluster.append("yay")
        # else: 
            # print("nay..")
    window_clusters.append(cluster)  

print(window_clusters)  

[0, ['yay', 'yay', 'yay'], 1, ['yay', 'yay', 'yay', 'yay', 'yay', 'yay'], 2, ['yay', 'yay', 'yay'], 3, ['yay', 'yay', 'yay', 'yay'], 4, ['yay', 'yay', 'yay', 'yay'], 5, ['yay', 'yay', 'yay', 'yay'], 6, ['yay', 'yay', 'yay'], 7, ['yay', 'yay', 'yay'], 8, ['yay', 'yay', 'yay'], 9, ['yay']]


In [124]:
# chatgpt code, messsssss
class UnionFind:
    def __init__(self, size):
        self.parent = list(range(size))
        self.rank = [1] * size
    
    def find(self, node):
        if self.parent[node] != node:
            self.parent[node] = self.find(self.parent[node])
        return self.parent[node]
    
    def union(self, node1, node2):
        root1 = self.find(node1)
        root2 = self.find(node2)
        
        if root1 != root2:
            if self.rank[root1] > self.rank[root2]:
                self.parent[root2] = root1
            elif self.rank[root1] < self.rank[root2]:
                self.parent[root1] = root2
            else:
                self.parent[root2] = root1
                self.rank[root1] += 1

def is_overlapping(interval1, interval2):
    return interval1[1] >= interval2[0] and interval2[1] >= interval1[0]

def cluster_intervals(intervals):
    n = len(intervals)
    uf = UnionFind(n)
    
    for i in range(n):
        for j in range(i + 1, n):
            if is_overlapping(intervals[i], intervals[j]):
                uf.union(i, j)
    
    # Count clusters
    root_counter = {}
    for i in range(n):
        root = uf.find(i)
        if root in root_counter:
            root_counter[root] += 1
        else:
            root_counter[root] = 1
    
    # Number of clusters and sets per cluster
    num_clusters = len(root_counter)
    sets_per_cluster = list(root_counter.values())
    
    return num_clusters, sets_per_cluster

# Example usage
y_bounds = [[738, 665], [735, 658], [733, 705], [661, 603], [660, 602], [658, 596], [588, 540], [586, 538], [584, 532], [526, 484]]
num_clusters, sets_per_cluster = cluster_intervals(y_bounds)
print(f"Number of clusters: {num_clusters}")
print(f"Sets per cluster: {sets_per_cluster}")

Number of clusters: 10
Sets per cluster: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [103]:
# this one is working as a backup
for second_window in y_bounds: 
    for window in y_bounds: 
        if window[1] <= second_window[0] and second_window[1] <= window[0]: 
            print("yay!")
        else: 
            print("nay..")    

[738, 665]
yay!
[735, 658]
yay!
[733, 705]
yay!
[661, 603]
nay..
[660, 602]
nay..
[658, 596]
nay..
[588, 540]
nay..
[586, 538]
nay..
[584, 532]
nay..
[526, 484]
nay..
[738, 665]
yay!
[735, 658]
yay!
[733, 705]
yay!
[661, 603]
yay!
[660, 602]
yay!
[658, 596]
yay!
[588, 540]
nay..
[586, 538]
nay..
[584, 532]
nay..
[526, 484]
nay..
[738, 665]
yay!
[735, 658]
yay!
[733, 705]
yay!
[661, 603]
nay..
[660, 602]
nay..
[658, 596]
nay..
[588, 540]
nay..
[586, 538]
nay..
[584, 532]
nay..
[526, 484]
nay..
[738, 665]
nay..
[735, 658]
yay!
[733, 705]
nay..
[661, 603]
yay!
[660, 602]
yay!
[658, 596]
yay!
[588, 540]
nay..
[586, 538]
nay..
[584, 532]
nay..
[526, 484]
nay..
[738, 665]
nay..
[735, 658]
yay!
[733, 705]
nay..
[661, 603]
yay!
[660, 602]
yay!
[658, 596]
yay!
[588, 540]
nay..
[586, 538]
nay..
[584, 532]
nay..
[526, 484]
nay..
[738, 665]
nay..
[735, 658]
yay!
[733, 705]
nay..
[661, 603]
yay!
[660, 602]
yay!
[658, 596]
yay!
[588, 540]
nay..
[586, 538]
nay..
[584, 532]
nay..
[526, 484]
nay..
[738

In [ ]:
# for each 
# if x_start <= y_end && y_start <= x_end

In [ ]:
# # Display the result
# cv2.imshow('Blue Squares and Borders', contours)
# cv2.waitKey(0)
# cv2.destroyAllWindows()